In [101]:
from datetime import datetime as dt
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.nn import Sequential, Conv2d, MaxPool2d, Linear, ReLU, CrossEntropyLoss
from torch.optim import Adam
from torchvision.transforms import ToTensor
import torch

In [102]:
train = MNIST("/files", train=True, transform=ToTensor(), download=True)
test = MNIST("/files", train=False, transform=ToTensor(), download=True)

In [103]:
loaders = {"train":DataLoader(train, shuffle=True, batch_size=64, num_workers=4), "test":DataLoader(test, shuffle=True, batch_size=64, num_workers=4)}

In [104]:
class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.conv1 = Sequential(Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),ReLU(),MaxPool2d(kernel_size=2))
        self.conv2 = Sequential(Conv2d(16,32,5,1,2),ReLU(),MaxPool2d(2))
        self.out = Sequential(Linear(32 * 7 * 7, 10))
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1)
        output = self.out(x)
        
        return output, x

In [105]:
cnn = NeuralNetwork()
print(cnn)
print(f"Network printed at {dt.now()}")

NeuralNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Sequential(
    (0): Linear(in_features=1568, out_features=10, bias=True)
  )
)
Network printed at 2022-04-19 20:51:40.624818


In [106]:
adam = Adam(cnn.parameters(), lr=0.01)
loss_function = CrossEntropyLoss()
n_epochs = 10

In [109]:
def train(num_epochs, cnn, loaders):
    
    cnn.train()
    
    total_steps = len(loaders["train"])
    
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders["train"]):
            b_x = Variable(images)
            b_y = Variable(labels)
            output = cnn(b_x)[0]
            loss = loss_function(output, b_y)
            
            adam.zero_grad()
            
            loss.backward()
            
            adam.step()
            
            if (i+1) % 100 == 0:
                print(f"Epoch: {epoch+1}/{num_epochs} | Loss: {loss.item()}")

In [110]:
train(n_epochs, cnn, loaders)
print(f"Training started at {dt.now()}")

Epoch: 1/10 | Loss: 0.16494667530059814
Epoch: 1/10 | Loss: 0.05170709639787674
Epoch: 1/10 | Loss: 0.03412928432226181
Epoch: 1/10 | Loss: 0.1400936096906662
Epoch: 1/10 | Loss: 0.12785311043262482
Epoch: 1/10 | Loss: 0.2201474905014038
Epoch: 1/10 | Loss: 0.16198056936264038
Epoch: 1/10 | Loss: 0.03369157388806343
Epoch: 1/10 | Loss: 0.11614373326301575
Epoch: 2/10 | Loss: 0.12191595882177353
Epoch: 2/10 | Loss: 0.008380996063351631
Epoch: 2/10 | Loss: 0.09810294210910797
Epoch: 2/10 | Loss: 0.00663779117166996
Epoch: 2/10 | Loss: 0.025399595499038696
Epoch: 2/10 | Loss: 0.1259947270154953
Epoch: 2/10 | Loss: 0.004791941959410906
Epoch: 2/10 | Loss: 0.01198610384017229
Epoch: 2/10 | Loss: 0.2144889384508133
Epoch: 3/10 | Loss: 0.1494968980550766
Epoch: 3/10 | Loss: 0.2969485819339752
Epoch: 3/10 | Loss: 0.0031733682844787836
Epoch: 3/10 | Loss: 0.043931130319833755
Epoch: 3/10 | Loss: 0.1367957890033722
Epoch: 3/10 | Loss: 0.014337446540594101
Epoch: 3/10 | Loss: 0.012439067475497723

In [116]:
def test():
    
    cnn.eval()
    
    with torch.no_grad():
        correct = 0
        total = 0
        
        for images, labels in loaders["test"]:
            test_output, last_layer = cnn(images)
            y_predictions = torch.max(test_output,1)[1].data.squeeze()
            accuracy = (y_predictions == labels).sum().item() / float(labels.size(0))
            pass
        
        print(f"Test Accuracy on 10,000 Images: {accuracy}")

In [117]:
test()

Test Accuracy on 10,000 Images: 1.0


In [118]:
sample = next(iter(loaders["test"]))
images, labels = sample
actual_number = labels[:10].numpy()
test_output, last_layer = cnn(images[:10])
y_predictions = torch.max(test_output,1)[1].data.numpy().squeeze()

In [119]:
print(f"Predictions Made at {dt.now()}")
print("Predicted Number: {} | Real Number: {}".format(y_predictions, actual_number))

Predictions Made at 2022-04-19 20:57:34.848959
Predicted Number: [2 9 7 5 6 9 9 1 3 7] | Real Number: [2 9 7 5 6 9 9 1 3 7]
